In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

mydriver= webdriver.Chrome() 
#change category and category link
category="Boys 9-14Y"
mydriver.get(r"https://eg.hm.com/en/shop-kids/boys-9-14y/view-all/--presentation_product_type-blazer-blouse-bodysuit-cardigan-coat-costume-dress-dressing__gown-gilet-jacket-jeans-jumper-leggings-night__dress-pyjamas-shirt-shorts-skirt-sweatshirt-swim__shorts-swim__top-swimsuit-t_shirt-top-trousers-waist__coat")

In [8]:
while True:
    try:
        # Wait for the "Next" button to be clickable
        next_button = WebDriverWait(mydriver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='pager-button']"))
        )
        
        # Scroll the button into view
        actions = ActionChains(mydriver)
        actions.move_to_element(next_button).perform()
        
        # Add a small delay
        time.sleep(1)
        
        # Click the button using JavaScript
        mydriver.execute_script("arguments[0].click();", next_button)
        
        # Wait for the page to load
        time.sleep(2)
        
    except ElementClickInterceptedException:
        print("Element is still obstructed. Retrying...")
        continue  # Retry clicking the button
        
    except (NoSuchElementException, TimeoutException):
        print("No more pages to load or 'Next' button not found.")
        break  # Exit the loop if the "Next" button is not found


No more pages to load or 'Next' button not found.


In [9]:
items_links=[]
#page items:
my_elems = mydriver.find_elements("xpath","//div[@class='product-item card']")

for i in range(len(my_elems)):
        items_links.append(my_elems[i].find_element("xpath","a[@class='product-item-link']").get_attribute("href"))
print(len(items_links))
items_links

273


['https://eg.hm.com/en/buy-sports-jacket-drymovetm-navy-blue',
 'https://eg.hm.com/en/buy-drymovetm-sports-t-shirt-white-patterned',
 'https://eg.hm.com/en/buy-printed-t-shirt-white-make-moment',
 'https://eg.hm.com/en/buy-5-pack-knee-length-sweatshorts-black-light-blue',
 'https://eg.hm.com/en/buy-cargo-twill-shorts-khaki-green',
 'https://eg.hm.com/en/buy-5-pack-cotton-vest-tops-navy-blue-light-blue',
 'https://eg.hm.com/en/buy-cotton-canvas-parachute-trousers-black',
 'https://eg.hm.com/en/buy-cotton-canvas-parachute-trousers-green',
 'https://eg.hm.com/en/buy-3-pack-shorts-light-beige-brown',
 'https://eg.hm.com/en/buy-2-pack-brushed-inside-joggers-dusty-blue-light-blue',
 'https://eg.hm.com/en/buy-v-neck-sweatshirt-cream',
 'https://eg.hm.com/en/buy-3-piece-dressy-set-light-beige-white-0',
 'https://eg.hm.com/en/buy-zip-top-polo-shirt-bright-blue',
 'https://eg.hm.com/en/buy-2-pack-baggy-fit-jeans-dark-grey-light-denim-blue',
 'https://eg.hm.com/en/buy-loose-fit-cotton-t-shirt-whi

In [10]:
# Create an empty DataFrame
df = pd.DataFrame(columns=["item title", "original price", "price on sale", "rating", "color options", "size options", "description", "Age Group", "Garment Length", "Presentation Product Type", "Sleeve Length", "Composition", "category"])
item_id=0
while(item_id<len(items_links)):
    try:
        print(f"******************************item {item_id}***************************")
        link=items_links[item_id]
        #go to link
        mydriver.get(link)

        # Wait until the document is ready
        time.sleep(2)

        data=[]

        #product_title
        data.append(mydriver.find_element("xpath", "//h6[@class='pdp-product__title']").text)

        #price before
        try:
            data.append(mydriver.find_element("xpath","//div[@class='pdp-product__prices']//span[@class='dropin-price dropin-price--small dropin-price--bold dropin-price--strikethrough']").text)
        except NoSuchElementException:
            data.append(np.nan)

        #price after
        data.append(mydriver.find_element("xpath","//div[@class='pdp-product__prices']//span[@class='dropin-price dropin-price--default dropin-price--small dropin-price--bold']").text)

        #rating:a class="pdp-product__ratings--count"
        try:
            mydriver.find_element("xpath","//a[@class='pdp-product__ratings--count']").click()
        except NoSuchElementException:
            data.append(np.nan)
        else:
            data.append(mydriver.find_element("xpath","//span[@class='pdp-product__ratings--average']").text)
            mydriver.find_elements("xpath","//div[@class='modal-header']/button[@class='modal-close']/span[@class='icon icon-close']/img[@src='/icons/close.svg']")[-1].click()

        #available colors:
        colors=[]
        my_elems = mydriver.find_elements("xpath", "//div[@class='pdp-swatches-list-container']/div[@class='pdp-swatches-refs']/a")
        for i in range(len(my_elems)):
            colors.append(my_elems[i].get_attribute('data-swatch-title'))
        data.append(colors)

        #available sizes:
        sizes=[]
        my_elems = mydriver.find_elements("xpath", "//div[@class='pdp-swatches__options']//label[@class='dropin-text-swatch__label']")
        for i in range(len(my_elems)):
            sizes.append(my_elems[i].text)
        data.append(sizes)

        #description section
        mydriver.find_element("xpath", "//h3[@class='pdp-product__description--title']").click()
        #description content
        data.append(mydriver.find_element("xpath", "//div[@class='pdp-product__description--content']").text)
        #description attributes:
        attributes={"Age Group":np.nan,"Garment Length":np.nan,"Presentation Product Type":np.nan,"Sleeve Length":np.nan,"Composition":np.nan}
        my_elems = mydriver.find_elements("xpath", "//div[@class='pdp-product-description__attributes']/ul[@class='pdp-product-description__attribute']/li")
        for i in range(len(my_elems)):
            attribute=my_elems[i].find_element(By.XPATH,".//div").text
            if attribute in attributes.keys():
                attributes[attribute]=my_elems[i].find_element(By.XPATH,".//span").text
        data=data+list(attributes.values())
        data.append(category)
        # Add data to DataFrame
        df.loc[len(df)] = data
        print(data)
        item_id+=1
    except:
        print(f"try again item {item_id}")

# Save the DataFrame to a CSV file (optional)
df.to_csv(category+".csv", index=False)

******************************item 0***************************
['Sports jacket in DryMove™', nan, 'EGP 2,799.00', nan, ['Navy blue'], ['10Y', '12Y', '14Y', '14Y+'], 'Regular-fit sports jacket in DryMove™ functional jersey that helps to wick away moisture from your skin, keeping you comfortably dry while you move. Hood, a zip down the front with an anti-chafe chin guard, raglan sleeves, zipped side pockets and elasticated binding at the cuffs and hem. Unlined.', 'Younger child, Older child', 'Regular length', 'Jacket, Sweatshirt', 'Long sleeve', '["57.0% Polyester", "43.0% Cotton"]', 'Boys 9-14Y']
******************************item 1***************************
['DryMove™ Sports T-shirt', nan, 'EGP 749.00', nan, ['Light blue/Ombre', 'White/Patterned'], ['10-12Y', '12-14Y', '14Y+'], 'Sports T-shirt in DryMove™ functional fabric that helps to wick away moisture from your skin, keeping you comfortably dry while you move. Regular fit with a round neckline and raglan sleeves to minimise the 

### Data Combining

In [ ]:
import pandas as pd
import os

folder_path = r"D:\Soha\ITI\Data Exploration and Preparation\Project"

#find all csv files in the path
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

#convert csv files to data frames
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]

#Combine all data in one frame
combined_df = pd.concat(df_list, ignore_index=True)

#Save as new csv file
combined_df.to_csv("Combined_Data.csv", index=False)

print("All CSV files have been combined successfully!")

All CSV files have been combined successfully!


### Data Exploring

In [43]:
combined_df = pd.read_csv("Combined_Data.csv")

In [44]:
combined_df.shape

(4535, 13)

In [46]:
combined_df.head()

,item title,original price,price on sale,rating,color options,size options,description,Age Group,Garment Length,Presentation Product Type,Sleeve Length,Composition,category
0,UPF 50 Sun protection set,NaN,"EGP 1,999.00",NaN,"['Light blue/Striped', 'Cream/Fish', 'Cream/Fl...","['1.5-2Y', '3-4Y', '4-5Y']",Sun protection suit and matching sun protectio...,Baby,Short,Swimsuit,Long sleeve,"[""83.0% Polyester"", ""17.0% Elastane""]",Baby Boy
1,Printed cotton T-shirt,NaN,EGP 449.00,NaN,[],"['6M', '9M', '12M', '18M', '2Y', '3Y', '4Y']",T-shirt in soft cotton jersey with print motif...,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy
2,Printed cotton T-shirt,NaN,EGP 449.00,NaN,"['Light blue/Striped', 'Yellow/California', 'T...","['18M', '2Y', '4Y']",T-shirt in soft cotton jersey with print motif...,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy
3,Printed cotton T-shirt,NaN,EGP 449.00,NaN,"['Light blue/Striped', 'Yellow/California', 'T...","['6M', '9M', '12M', '18M', '2Y', '3Y', '4Y']",T-shirt in soft cotton jersey with print motif...,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy
4,Long-sleeved bodysuit,NaN,EGP 399.00,NaN,"['White/Cherries', 'Light beige/Dogs', 'Light ...","['1.5-2Y', '12-18M', '4-6M', '6-9M', '9-12M']",Long-sleeved bodysuit in supersoft cotton jers...,Baby,NaN,Bodysuit,Long sleeve,"[""100.0% Cotton""]",Baby Boy


In [47]:
combined_df.tail()

,item title,original price,price on sale,rating,color options,size options,description,Age Group,Garment Length,Presentation Product Type,Sleeve Length,Composition,category
4530,Ribbed modal-blend T-shirt,NaN,EGP 749.00,NaN,"['Dark brown', 'Black', 'White', 'Light khaki ...","['XS', 'M', 'L', 'XL']","Fitted T-shirt in soft, ribbed cotton and moda...",Adult,Regular length,T-shirt,Short sleeve,"[""50% Cotton"", ""50% Modal""]",Women
4531,Knitted jumper,NaN,"EGP 1,099.00",NaN,"['Black/Striped', 'Cream/Striped', 'Navy blue'...",['XXS'],Jumper in a soft knit containing some wool wit...,Adult,Regular length,Jumper,Long sleeve,"[""41% Acrylic"", ""3% Elastane"", ""52% Polyester""...",Women
4532,Wide trousers,NaN,"EGP 2,499.00",NaN,"['Grey marl', 'Khaki green', 'Grey', 'Light mo...","['10', '14']","Tailored, loose-fitting trousers in woven fabr...",Adult,Long,Trousers,NaN,"[""4% Elastane"", ""96% Polyester""]",Women
4533,Cotton-blend shirt,NaN,"EGP 1,099.00",NaN,['White'],"['XXS', 'S', '3XL']",Long-sleeved shirt woven in a cotton blend wit...,Adult,NaN,Shirt,Long sleeve,"[""73% Cotton"", ""4% Elastane"", ""23% Polyester""]",Women
4534,Cotton T-shirt,NaN,EGP 499.00,NaN,"['Dark grey', 'White', 'Beige', 'Grey-blue', '...","['XS', 'M']",Straight-cut T-shirt in soft cotton jersey wit...,Adult,NaN,"Shorts, T-shirt",Short sleeve,"[""100% Cotton""]",Women


In [48]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4535 entries, 0 to 4534
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   item title                 4535 non-null   object 
 1   original price             0 non-null      float64
 2   price on sale              4535 non-null   object 
 3   rating                     0 non-null      float64
 4   color options              4535 non-null   object 
 5   size options               4535 non-null   object 
 6   description                4534 non-null   object 
 7   Age Group                  4387 non-null   object 
 8   Garment Length             3884 non-null   object 
 9   Presentation Product Type  4534 non-null   object 
 10  Sleeve Length              2295 non-null   object 
 11  Composition                3331 non-null   object 
 12  category                   4535 non-null   object 
dtypes: float64(2), object(11)
memory usage: 460.7+ K

In [49]:
#Check nulls
combined_df.isnull().sum()


item title                      0
original price               4535
price on sale                   0
rating                       4535
color options                   0
size options                    0
description                     1
Age Group                     148
Garment Length                651
Presentation Product Type       1
Sleeve Length                2240
Composition                  1204
category                        0
dtype: int64

In [50]:
# Check duplicate rows
combined_df.duplicated().sum()

np.int64(324)

#### *Notes on data after exploration till now*

- A lot of columns need to be prepared and encoded
- drop dulpicate rows
- original price column is all none (But there is sale price that can be the target, it is the same as the original if there is no sale) -> drop
- rating column is all none -> drop 
- color options, size options and description (do not affect) -> drop
- Garment Length may be effective but has sone nulls -> decision later
- Sleeve Length (Highly percent it will be dropped because not all items has sleeves and there is alot of nulls) -> decision later
- item title, Age Group, Presentation Product Type, Composition and category are the most effective columns *as it seems*, but need a lot of preparation

### Data Cleaning

In [51]:
# Moving target to the end
price = combined_df.pop("price on sale")
combined_df["price"] = price

In [52]:
combined_df.head(3)

,item title,original price,rating,color options,size options,description,Age Group,Garment Length,Presentation Product Type,Sleeve Length,Composition,category,price
0,UPF 50 Sun protection set,NaN,NaN,"['Light blue/Striped', 'Cream/Fish', 'Cream/Fl...","['1.5-2Y', '3-4Y', '4-5Y']",Sun protection suit and matching sun protectio...,Baby,Short,Swimsuit,Long sleeve,"[""83.0% Polyester"", ""17.0% Elastane""]",Baby Boy,"EGP 1,999.00"
1,Printed cotton T-shirt,NaN,NaN,[],"['6M', '9M', '12M', '18M', '2Y', '3Y', '4Y']",T-shirt in soft cotton jersey with print motif...,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy,EGP 449.00
2,Printed cotton T-shirt,NaN,NaN,"['Light blue/Striped', 'Yellow/California', 'T...","['18M', '2Y', '4Y']",T-shirt in soft cotton jersey with print motif...,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy,EGP 449.00


In [53]:
#Dropping columns that we are sure it must be dropped
combined_df.drop(["original price", "rating", "color options", "size options", "description"], axis=1, inplace=True)

In [54]:
combined_df.head(3)

,item title,Age Group,Garment Length,Presentation Product Type,Sleeve Length,Composition,category,price
0,UPF 50 Sun protection set,Baby,Short,Swimsuit,Long sleeve,"[""83.0% Polyester"", ""17.0% Elastane""]",Baby Boy,"EGP 1,999.00"
1,Printed cotton T-shirt,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy,EGP 449.00
2,Printed cotton T-shirt,Baby,Regular length,T-shirt,Short sleeve,"[""100.0% Cotton""]",Baby Boy,EGP 449.00


In [55]:
combined_df.duplicated().sum()

np.int64(1582)

In [56]:
#Dropping duplicate rows
combined_df.drop_duplicates(inplace=True)


In [57]:
combined_df.shape

(2953, 8)

In [59]:
combined_df.isnull().sum()

item title                      0
Age Group                     105
Garment Length                404
Presentation Product Type       1
Sleeve Length                1527
Composition                   765
category                        0
price                           0
dtype: int64

### Data Preparation